In [1]:
import pandas as pd 
import numpy as np
from scipy import stats
from statsmodels.stats.multitest import multipletests

In [30]:
def write_answer(answer, number):
    file_name = 'answer' + str(number) + '.txt'
    with open(file_name, 'w') as f:
        f.write(str(answer))
    print('answer', number, ': ', answer)

In [2]:
gen = pd.read_csv('gene_high_throughput_sequencing.csv')

In [7]:
gen.head()

,Patient_id,Diagnosis,LOC643837,LOC100130417,SAMD11,NOC2L,KLHL17,PLEKHN1,C1orf170,HES4,...,CLIC2,RPS4Y1,ZFY,PRKY,USP9Y,DDX3Y,CD24,CYorf15B,KDM5D,EIF1AY
0,STT5425_Breast_001_normal,normal,1.257614,2.408148,13.368622,9.494779,20.880435,12.722017,9.494779,54.349694,...,4.761250,1.257614,1.257614,1.257614,1.257614,1.257614,23.268694,1.257614,1.257614,1.257614
1,STT5427_Breast_023_normal,normal,4.567931,16.602734,42.477752,25.562376,23.221137,11.622386,14.330573,72.445474,...,6.871902,1.815112,1.815112,1.815112,1.815112,1.815112,10.427023,1.815112,1.815112,1.815112
2,STT5430_Breast_002_normal,normal,2.077597,3.978294,12.863214,13.728915,14.543176,14.141907,6.232790,57.011005,...,7.096343,2.077597,2.077597,2.077597,2.077597,2.077597,22.344226,2.077597,2.077597,2.077597
3,STT5439_Breast_003_normal,normal,2.066576,8.520713,14.466035,7.823932,8.520713,2.066576,10.870009,53.292034,...,5.200770,2.066576,2.066576,2.066576,2.066576,2.066576,49.295538,2.066576,2.066576,2.066576
4,STT5441_Breast_004_normal,normal,2.613616,3.434965,12.682222,10.543189,26.688686,12.484822,1.364917,67.140393,...,11.227770,1.364917,1.364917,1.364917,1.364917,1.364917,23.627911,1.364917,1.364917,1.364917


In [8]:
normal = gen[gen['Diagnosis']=='normal'].drop(['Patient_id', 'Diagnosis'], axis=1)
early_n = gen[gen['Diagnosis']=='early neoplasia'].drop(['Patient_id', 'Diagnosis'], axis=1)
cancer = gen[gen['Diagnosis']=='cancer'].drop(['Patient_id', 'Diagnosis'], axis=1)

### Часть 1: применение t-критерия Стьюдента

В первой части вам нужно будет применить критерий Стьюдента для проверки гипотезы о равенстве средних в двух независимых выборках. Применить критерий для каждого гена нужно будет дважды:

для групп normal (control) и early neoplasia (treatment)
для групп early neoplasia (control) и cancer (treatment)
В качестве ответа в этой части задания необходимо указать количество статистически значимых отличий, которые вы нашли с помощью t-критерия Стьюдента, то есть число генов, у которых p-value этого теста оказался меньше, чем уровень значимости.



In [9]:
def comp_samles(a, b):
    p_val = []
    for i, j in zip(np.array(a.T), np.array(b.T)):
        p_val.append(stats.ttest_ind(i, j, equal_var=False).pvalue)
    return np.array(p_val)

In [10]:
def count_rel_feat(data, a=0.05):
    return np.sum(data < a)

In [42]:
p_ne = comp_samles(normal, early_n)
p_ec = comp_samles(early_n, cancer)

In [43]:
write_answer(count_rel_feat(p_ne), 1)

answer 1 :  1575


In [44]:
write_answer(count_rel_feat(p_ec), 2)

answer 2 :  3490


In [236]:
#def t_st(a, b):
#    S = lambda x: np.sqrt(np.sum([(i - x.mean())**2 for i in x]) / len(x))
#    t = (a.mean() - b.mean()) / np.sqrt((S(a)**2 / len(a) + S(b)**2 / len(b)))
#    return stats.t.cdf(t)

### Часть 2: поправка методом Холма

Для этой части задания вам понадобится модуль multitest из statsmodels.


В этой части задания нужно будет применить поправку Холма для получившихся двух наборов достигаемых уровней значимости из предыдущей части. Обратите внимание, что поскольку вы будете делать поправку для каждого из двух наборов p-value отдельно, то проблема, связанная с множественной проверкой останется.

Для того, чтобы ее устранить, достаточно воспользоваться поправкой Бонферрони, то есть использовать уровень значимости 0.05 / 2 вместо 0.05 для дальнейшего уточнения значений p-value c помощью метода Холма.

В качестве ответа к этому заданию требуется ввести количество значимых отличий в каждой группе после того, как произведена коррекция Холма-Бонферрони. Причем это число нужно ввести с учетом практической значимости: посчитайте для каждого значимого изменения fold change и выпишите в ответ число таких значимых изменений, абсолютное значение fold change которых больше, чем 1.51.5.

Обратите внимание, что

применять поправку на множественную проверку нужно ко всем значениям достигаемых уровней значимости, а не только для тех, которые меньше значения уровня доверия.
при использовании поправки на уровне значимости 0.025 меняются значения достигаемого уровня значимости, но не меняется значение уровня доверия (то есть для отбора значимых изменений скорректированные значения уровня значимости нужно сравнивать с порогом 0.025, а не 0.05)!

In [69]:
p_ne_holm = multipletests(p_ne, alpha=0.025, method='holm')[1]
p_ec_holm = multipletests(p_ec, alpha=0.025, method='holm')[1]

In [70]:
count_rel_feat(multipletests(p_ne, alpha=0.025, method='holm')[1], 0.025)

2

In [71]:
count_rel_feat(multipletests(p_ec, alpha=0.025, method='holm')[1], 0.025)

79

In [59]:
def fold_change(c, t, limit=1.5):
    C, T = c.mean(), t.mean()
    
    if T >= C:
        fc_stat = T / C
    else:
        fc_stat = -C / T
        
    return (np.abs(fc_stat) > limit), fc_stat

In [65]:
def count_rel_feat_FC(a, b, p):
    rel_feat_count = 0
    for c, t, p in zip(np.array(a.T), np.array(b.T), p):  
        if fold_change(c, t)[0]==True and p < 0.025:        
            rel_feat_count += 1
    return rel_feat_count

In [67]:
write_answer(count_rel_feat_FC(normal, early_n, p_ne_holm), 3)

answer 3 :  2


In [72]:
write_answer(count_rel_feat_FC(early_n, cancer, p_ec_holm), 4)

answer 4 :  77


### Часть 3: поправка методом Бенджамини-Хохберга
    
Данная часть задания аналогична второй части за исключением того, что нужно будет использовать метод Бенджамини-Хохберга.

Обратите внимание, что методы коррекции, которые контролируют FDR, допускает больше ошибок первого рода и имеют большую мощность, чем методы, контролирующие FWER. Большая мощность означает, что эти методы будут совершать меньше ошибок второго рода (то есть будут лучше улавливать отклонения от H_0, когда они есть, и будут чаще отклонять H_0, когда отличий нет).

В качестве ответа к этому заданию требуется ввести количество значимых отличий в каждой группе после того, как произведена коррекция Бенджамини-Хохберга, причем так же, как и во второй части, считать только такие отличия, у которых abs(fold change) > 1.5.

In [73]:
p_ne_fdr_bh = multipletests(p_ne, alpha=0.025, method='fdr_bh')[1]
p_ec_fdr_bh = multipletests(p_ec, alpha=0.025, method='fdr_bh')[1]

In [74]:
write_answer(count_rel_feat_FC(normal, early_n, p_ne_fdr_bh), 5)

answer 5 :  4


In [75]:
write_answer(count_rel_feat_FC(early_n, cancer, p_ec_fdr_bh), 6)

answer 6 :  524
